## 프로시저 언어
- 프로시저 언어는 함수와 트리거를 만드는데 사용한다. 
- PostgreSQL의 프로시저 언어로는 PL/pgSQL, PL/TCL, PL/Perl, PL/Python 등이 존재함. 
- 프로시저 언어를 추가 시, DB에 접속 후 다음과 같이 입력하면 된다. 

```SQL
CREATE LANGUAGE 언어이름
```
- 참고자료: [Chapter 39. Procedural Languages](https://www.postgresql.org/docs/9.4/xplang.html)

## PostgreSQL에서 함수 사용하기
- 간단한 함수를 만들도록 한다. 
- 함수를 만드는 방식은 아래와 같다. 

```SQL
CREATE FUNCTION function_name(parameter_1 type, parameter_2 type)
    RETURN return_type AS
    $$ BEGIN 
    ...
    END; $$
LANGUAGE language_name;
```

- 곱셈 함수를 만들어 본다. 

```SQL
CREATE FUNCTION multiple(a INTEGER, b INTEGER)
RETURNS INTEGER AS
$$ BEGIN
    RETURN a * b;
END; $$
LANGUAGE PLpgSQL;
```
```bash
postgres=# \df
                                                  List of functions
 Schema |          Name          |      Result data type      |              Argument data types               | Type 
--------+------------------------+----------------------------+------------------------------------------------+------
 public | connectby              | SETOF record               | text, text, text, text, integer                | func
 public | multiple               | integer                    | a integer, b integer                           | func
 
- 이제 실제로 불러와서 사용하도록 합니다. 

```SQL
postgres=# SELECT multiple(3, 4);
 multiple 
----------
       12
(1 row)
```

- 위와 같이 함수가 정확하게 작동하는 것을 볼 수 있습니다. 




## 트리거의 개념
- 예) 유투브에서 구독자가 구독 버튼을 누르면 자동으로 구독자수가 갱신되는 시스템
    + 어떠한 행동이나 작업을 했을 때 미리 정해놓은 작업이 자동으로 실행되도록 하는 것을 "트리거"라고 함. 

### PostgreSQL에서 트리거 사용하기
- 구독자가 삽입(insert) 되면 구독자수를 늘리는 트리거를 만들어 본다. 
- 먼저 subs_table을 작성한다. 
- 두번째 테이블인 num_subs을 작성한다. 

```SQL
DROP TABLE subs_table, num_subs;

CREATE TABLE subs_table(
    subs_id INTEGER, 
    subs_name VARCHAR(80));
	
CREATE TABLE num_subs(
	subs_num INTEGER);
```

- 구독자수는 처음에 0이므로 0을 넣어준다. 

```SQL
INSERT INTO num_subs VALUEs(0);
```
```
INSERT 0 1
```

- 이번에는 구독자수를 늘려주는 함수를 만든다. 
- 함수의 이름을 inc()로 정하고, 구독자수 1을 늘리는 UPDATE문을 입력한다. 
- 언어는 PL/pgSQL로 한다. 

```SQL
CREATE FUNCTION inc()
RETURNS TRIGGER AS
$$ BEGIN
UPDATE num_subs SET subs_num = subs_num + 1;
RETURN NULL;
END; $$
LANGUAGE PLpgSQL;
```
```
CREATE FUNCTION
```

- 이번에는 트리거를 만들 차례이다. 함수를 만들때와 마찬가지로 트리거 실행을 의미하는 CREATE TRIGGER 명령어 뒤에 만들 트리거의 이름을 입력한다. 
- 구독자가 테이블에 INSERT 된 후 트리거를 실행해야 하기 때문에, AFTER INSERT ON 테이블의 형태로 코드를 입력한다. 
- 트리거 실행 시, 실행할 함수 inc() 수행하는 코드를 입력한다. 

```SQL
CREATE TRIGGER subs_insert AFTER INSERT ON subs_table
FOR EACH ROW EXECUTE PROCEDURE inc();
```
```
CREATE TRIGGER
```

- 트리거를 실행할 환경이 끝났기 때문에, subs_table에 임의의 값을 입력하도록 한다. 
```SQL
INSERT INTO subs_table VALUES
(1, 'a'), 
(2, 'b'), 
(3, 'c'); 
```
```
INSERT 0 3
```

- 그리고 num_subs 테이블을 조회해본다. 
```SQL
SELECT * FROM num_subs;
```
```
 subs_num 
----------
        3
(1 row)
```

- 구독자 3명을 넣었기 때문에 결괏값이 제대로 나온 것을 확인할 수 있다. 

